In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col

# Initialize Spark session however it's appropriate
spark = SparkSession.builder.appName("FlattenJSON").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/01 15:54:49 WARN Utils: Your hostname, Pauls-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.99 instead (on interface en0)
26/02/01 15:54:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/01 15:54:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Tell Spark you want to read a file that is multiline
# Then read using .json(file_name)
df = (
    spark
    .read
    .option("multiline", "true")
    .json('./hospitals.json'))


In [3]:
# To flatten the structure, we "explode" the hospitals attribute into their own individual rows
# And slect the various columns that we want from the main row and from the hospital object
flattened_df = (df
    .withColumn("hospital", explode(col("hospitals")))
    .select(
        col("system"),
        col("hospital.name").alias("hospital_name"),
        col("hospital.city").alias("city"),
        col("hospital.beds").alias("beds")
    ))

# Show the result
flattened_df.show()



+------+-------------+-----------+----+
|system|hospital_name|       city|beds|
+------+-------------+-----------+----+
|   BJC|          BJH|  St. Louis|1432|
|   BJC|        MOBap|Creve Coeur|1107|
| Mercy|    Mercy STL|Creve Coeur| 983|
|   SSM|         SLUH|  St. Louis| 965|
+------+-------------+-----------+----+

